# XPS utilities

Several helper methods exist for XPS measurements, beyond the general [fitting functions](./6_data_fitting.ipynb).

In [ ]:
# Import packages
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import peaks as pks
import os

# Set default options
xr.set_options(cmap_sequential='Purples', keep_attrs=True)
%matplotlib inline
%config InlineBackend.figure_format='retina'

# Example data
from peaks.core.utils.sample_data import ExampleData

In [ ]:
# Load some example data (a TiSe2 sample measured at 200 eV)
spectrum = ExampleData.xps().DOS()
spectrum.plot()

The XPS utilities can be accessed via the `peaks.xps` module.

In [ ]:
# Import the xps module
from peaks import xps

## Core level energies
A dictionary of approximate core level binding energies is avaiable in `peaks.xps.CoreLevels.BE`. The data is based on https://userweb.jlab.org/~gwyn/ebindene.html, and no guarantee of accuracy is provided.

In [ ]:
xps.CoreLevels.BE['Se']

### Lookup by element
A quick lookup can by made by element(s). By default, the binding energy is returned, and if a photon energy is passed via the `hv` argument, the corresponding kinetic energy is also given. Optionally, the kinetic energy expected for core levels excited by higher-order light can also be provided by setting the `max_order` argument.

In [ ]:
xps.CoreLevels.by_element(['Ti','Se'], hv=200)

### Search by energy
In addition, core levels can also be searched by binding energy (if no `hv` argument is supplied) or by kinetic energy (if `hv` supplied, also optionally considering higher-order light):

In [ ]:
xps.CoreLevels.by_energy(140, hv=200, tol=1, max_order=2)

### Plotting core level energies
A helper function allows plotting the expected core level (binding/kinetic) energies. These can be plotted on an existing plot if specified, and optionally limited to a finite energy range. 

In [ ]:
spectrum.sel(eV=slice(100,180)).plot()
ax=plt.gca()
xps.CoreLevels.plot(['Ti','Se'], ax=ax, hv=200, eV=slice(125,175))

## Cross sections
A :class:`xarray.DataTree` contining core level cross section data from https://vuo.elettra.eu/services/elements/WebElements.html is available in `peaks:xps.CoreLevels.xc`

In [ ]:
xps.CoreLevels.xc.Se['3d'].data.plot(label='Se 3d')
xps.CoreLevels.xc.Ti['3s'].data.plot(label='Ti 3s')
plt.legend()
plt.axvline(200, color='k', linestyle='--')
plt.text(210, 0.2, 'hv=200 eV', rotation=90, verticalalignment='bottom', horizontalalignment='left', transform=plt.gca().get_xaxis_transform())

Our usual :class:`xarray.DataTree` methods can be used:

In [ ]:
xps.CoreLevels.xc.Se.view()

In [ ]:
xps.CoreLevels.xc.Se.plot_grid()

## Interactive view
An interactive periodic table view of the core level kinetic/binding energies can also be shown with the `.ptab` accessor. Hover above an element to get some basic information, including the core level energies.

In [ ]:
xps.CoreLevels.ptab(hv=200)